# Imports and Initial Setup

In [3]:
from ppt import *  # Assuming custom application logic and definitions are here
from plot_functions import *

# Corrected imports and initial setup
from plot_functions import plot_graph, plot_histogram  # Assuming plot functions are defined
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from PIL import Image

In [4]:
# Function to update the plot in the customization window dynamically
def update_sidebar():
    """Update the sidebar to show slide icons or labels for each slide."""
    sidebar_content = []
    for i, slide in enumerate(slides):
        slide_button = widgets.Button(description=f"Slide {i + 1}", layout=widgets.Layout(width='100%'))
        
        # Bind each button to update `selected_slide_index` and render that slide's plots
        slide_button.on_click(lambda b, idx=i: select_slide(idx))
        
        sidebar_content.append(slide_button)
    sidebar.children = sidebar_content



# Right Sidebar Customization
def open_customization_window():
    plot_type_dropdown = widgets.Dropdown(options=list(plot_registry.keys()), description="Plot Type")
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position")
    
    param_sliders = [
        widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="param1"),
        widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="param2"),
        widgets.ColorPicker(value="blue", description="color")
    ]
    
    preview_output = widgets.Output()
    add_button = widgets.Button(description="Add Plot")
    add_button.on_click(lambda b: add_plot(plot_type_dropdown.value, position_dropdown.value, param_sliders, preview_output))
    
    # Update the right sidebar with customization options and preview
    customization_window.children = [plot_type_dropdown, position_dropdown] + param_sliders + [preview_output, add_button]


# Select and render the chosen slide
def select_slide(index):
    global selected_slide_index
    selected_slide_index = index
    render_slide()

# Layout for the application
def display_app_layout():
    update_sidebar()
    render_slide()
    open_customization_window()
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))

def add_plot():
    """Display customization options for adding a new plot in the customization window."""
    plot_types = list(plot_registry.keys())
    plot_type_dropdown = widgets.Dropdown(options=plot_types, description="Plot Type:")
    
    # Position dropdown for selecting plot position
    position_options = ["top-left", "top-right", "bottom-left", "bottom-right"]
    position_dropdown = widgets.Dropdown(options=position_options, description="Position:")
    
    # Set up plot controls based on plot type
    controls = create_plot_controls(plot_type_dropdown.value)
    plot_output = widgets.Output()
    update_plot(plot_type_dropdown.value, plot_output, controls)
    
    # Observe plot type dropdown to update controls and plot preview on change
    plot_type_dropdown.observe(lambda change: update_plot_type(change, plot_type_dropdown, plot_output, controls), names="value")
    
    # Confirm Button for Adding the Plot
    confirm_button = widgets.Button(description="Confirm Plot Addition")
    confirm_button.on_click(lambda b: on_confirm_button_click(b, slides[selected_slide_index], plot_type_dropdown, position_dropdown, controls))

    # Display all customization widgets in the customization window
    customization_window.children = [plot_type_dropdown, position_dropdown] + controls + [plot_output, confirm_button]
    print("Displayed customization window for adding plot.")  # Debug print

def on_confirm_button_click(b, slide, plot_type_dropdown, position_dropdown, controls):
    """Add plot with selected parameters and position to the current slide."""
    plot_type = plot_type_dropdown.value
    position = position_dropdown.value  # Capture selected position
    params = {control.description: control.value for control in controls}
    new_plot = {
        "uid": f"plot_{len(slide['plots']) + 1}",
        "function_name": plot_type,
        "params": params,
        "position": position  # Store position in plot data
    }
    
    # Debugging: Print to confirm that a new plot is being added with position
    print(f"Adding plot to slide {slide['uid']} at {position}: {new_plot}")
    
    slide["plots"].append(new_plot)
    render_slide(selected_slide_index)  # Refresh the main display area


def get_position(position):
    """
    Returns the layout parameters for positioning based on the given layout position.
    """
    position_map = {
        "top-left": (0, 0),  # Top-left corner
        "top-right": (400, 0),  # Top-right corner
        "bottom-left": (0, 300),  # Bottom-left corner
        "bottom-right": (400, 300),  # Bottom-right corner
    }
    return position_map.get(position, (0, 0))


def add_plot():
    """Display customization options for adding a new plot on the selected slide."""
    plot_types = list(plot_registry.keys())
    plot_type_dropdown = widgets.Dropdown(options=plot_types, description="Plot Type:")
    
    # Position dropdown for selecting plot position
    position_options = ["top-left", "top-right", "bottom-left", "bottom-right"]
    position_dropdown = widgets.Dropdown(options=position_options, description="Position:")
    
    # Set up plot controls based on plot type
    controls = create_plot_controls(plot_type_dropdown.value)
    plot_output = widgets.Output()
    update_plot(plot_type_dropdown.value, plot_output, controls)
    
    # Observe plot type dropdown to update controls and plot preview on change
    plot_type_dropdown.observe(lambda change: update_plot_type(change, plot_type_dropdown, plot_output, controls), names="value")
    
    # Confirm Button for Adding the Plot
    confirm_button = widgets.Button(description="Confirm Plot Addition")
    confirm_button.on_click(lambda b: on_confirm_button_click(b, slides[selected_slide_index], plot_type_dropdown, position_dropdown, controls))

    # Display all customization widgets in the customization window
    customization_window.children = [plot_type_dropdown, position_dropdown] + controls + [plot_output, confirm_button]
    print(f"Displayed customization window for adding plot to Slide {selected_slide_index + 1}")  # Debug print




def render_slide(slide_index):
    """Display the selected slide's plots in the main slide display area."""
    main_slide_display.clear_output(wait=True)
    with main_slide_display:
        slide = slides[slide_index]
        print(f"Displaying Slide {slide_index + 1} with plots: {slide['plots']}")  # Debug print

        # Display each plot in the slide based on its position
        for plot_data in slide["plots"]:
            fig, ax = plt.subplots(figsize=(5, 4))
            plot_function = plot_registry[plot_data["function_name"]]["function"]
            params = plot_data["params"]
            print(f"Rendering plot with params: {params} at position: {plot_data['position']}")  # Debug print
            plot_function(ax, **params)
            
            # Adjust plot position based on selected location
            left, top = get_position(plot_data["position"])
            main_slide_display.layout = widgets.Layout(margin=f"{top}px 0 0 {left}px")  # Set layout for positioning
            plt.show()


def select_slide(slide_index):
    """Update the selected slide index and re-render the slide's plots."""
    global selected_slide_index
    selected_slide_index = slide_index  # Set the selected slide index
    print(f"Selected Slide {slide_index + 1}")  # Debug print
    render_slide(slide_index)  # Render the selected slide's plots

def get_position(position):
    """
    Returns the layout parameters for positioning based on the given layout position.
    """
    position_map = {
        "top-left": (0, 0),  # Top-left corner
        "top-right": (400, 0),  # Top-right corner
        "bottom-left": (0, 300),  # Bottom-left corner
        "bottom-right": (400, 300),  # Bottom-right corner
    }
    return position_map.get(position, (0, 0))

# Render Main Slide Display
def render_slide():
    main_slide_display.clear_output(wait=True)
    with main_slide_display:
        slide = slides[selected_slide_index]
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        positions = ["top-left", "top-right", "bottom-left", "bottom-right"]
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            plot_function = plot_registry[plot_data["function_name"]]["function"]
            plot_function(ax, **plot_data["params"])
            ax.set_title("Customized Graph")
        plt.show()

# Update Left Sidebar (Overview of All Slides)
def update_sidebar():
    """Function to refresh the left sidebar with slide buttons."""
    left_sidebar.children = tuple()  # Start with an empty tuple
    for i, slide in enumerate(slides):
        slide_button = widgets.Button(description=f"Slide {i+1}", layout=widgets.Layout(width='100%', height='50px'))
        slide_button.on_click(lambda b, idx=i: select_slide(idx))
        left_sidebar.children = tuple(left_sidebar.children + (slide_button,))  # Append as tuple

def select_slide(index):
    global selected_slide_index
    selected_slide_index = index
    render_slide()

def add_plot(plot_type, position, param_sliders, preview_output):
    params = {slider.description: slider.value for slider in param_sliders}
    new_plot = {"function_name": plot_type, "params": params, "position": position}
    
    # Add to current slide
    slides[selected_slide_index]["plots"].append(new_plot)
    
    # Preview the customized plot in the preview_output
    with preview_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_registry[plot_type]["function"](ax, **params)
        plt.show()
    
    render_slide()  # Update the main slide display

def open_customization_window(plot_type="plot_graph"):
    """Displays the customization window with controls and a live-updating plot."""
    global param1_slider, param2_slider, color_picker, plot_output, position_dropdown, plot_type_dropdown

    # Dropdown for plot type
    plot_type_dropdown = widgets.Dropdown(options=list(plot_registry.keys()), description="Plot Type:")
    
    # Dropdown for position selection
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"],
                                         description="Position")

    # Initialize sliders and color picker for parameters
    param1_slider = widgets.FloatSlider(value=plot_registry[plot_type]["params"].get("param1", 1.0), 
                                        min=0.1, max=2.0, step=0.1, description="param1")
    param2_slider = widgets.FloatSlider(value=plot_registry[plot_type]["params"].get("param2", 1.0), 
                                        min=0.1, max=2.0, step=0.1, description="param2")
    color_picker = widgets.ColorPicker(value=plot_registry[plot_type]["params"].get("color", "blue"), description="color")
    plot_output = widgets.Output()

    # Attach observers to dynamically update the plot in the customization window
    param1_slider.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")
    param2_slider.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")
    color_picker.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")
    plot_type_dropdown.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")

    # Initial plot rendering
    update_custom_plot(plot_type)

    # Confirm button to add plot to slide
    confirm_button = widgets.Button(description="Confirm Plot Addition")
    confirm_button.on_click(lambda b: add_plot_to_slide(position_dropdown.value))

    # Display the customization window with sliders, color picker, position dropdown, and live plot
    customization_window.children = [plot_type_dropdown, position_dropdown, param1_slider, param2_slider, color_picker, plot_output, confirm_button]

def update_custom_plot(plot_type):
    """Update the plot in the customization window based on current widget values."""
    # Get the function to plot from the registry
    plot_function = plot_registry[plot_type]["function"]

    # Collect the current parameter values
    param_values = {
        param1_slider.description: param1_slider.value,
        param2_slider.description: param2_slider.value,
        color_picker.description: color_picker.value
    }
    
    # Clear the output widget and display the updated plot
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_function(ax, **param_values)  # Pass **param_values to the function
        plt.show()

def add_plot_to_slide(position="top-left"):
    """Add a new plot to the selected slide based on the customization window inputs."""
    plot_type = plot_type_dropdown.value  # Get the selected plot type from the dropdown
    params = {
        "param1": param1_slider.value,
        "param2": param2_slider.value,
        "color": color_picker.value
    }
    new_plot = {
        "function_name": plot_type,  # Use the selected plot type
        "params": params,
        "position": position
    }
    slides[selected_slide_index]["plots"].append(new_plot)
    render_slide()  # Refresh the slide after adding the plot

def on_confirm_button_click(b, slide, plot_type_dropdown, position_dropdown, controls):
    """Add plot with selected parameters and position to the current slide."""
    plot_type = plot_type_dropdown.value  # Capture the current selected plot type
    position = position_dropdown.value  # Capture the selected position
    params = {control.description: control.value for control in controls}
    new_plot = {
        "uid": f"plot_{len(slide['plots']) + 1}",
        "function_name": plot_type,  # Store the correct plot type here
        "params": params,
        "position": position  # Store position in plot data
    }
    
    # Debugging: Print to confirm that a new plot is being added with position
    print(f"Adding plot to slide {slide['uid']} at {position}: {new_plot}")
    
    slide["plots"].append(new_plot)
    render_slide(selected_slide_index)  # Refresh the main display area for the selected slide

### Inputs

In [5]:
# Initialize slides list
slides = [{"uid": f"slide_{i+1}", "plots": []} for i in range(4)]  # Create a few slides to start
selected_slide_index = 0  # Track the currently selected slide

# Set up widgets for each area
left_sidebar = widgets.VBox(layout=widgets.Layout(width="20%", border="1px solid black"))
main_slide_display = widgets.Output(layout=widgets.Layout(width="60%", border="1px solid black"))
customization_window = widgets.VBox(layout=widgets.Layout(width="20%", border="1px solid black"))

# Plot Registry and Functions
plot_registry = {
    "plot_graph": {
        "function": lambda ax, **params: ax.plot(
            np.linspace(0, 10, 100),
            params.get("param1", 1.0) * np.sin(np.linspace(0, 10, 100)) + params.get("param2", 1.0) * np.cos(np.linspace(0, 10, 100)),
            color=params.get("color", "blue")
        ),
        "params": {"param1": 1.0, "param2": 1.0, "color": "blue"}
    },
    "plot_histogram": {
        "function": lambda ax, **params: ax.hist(
            np.random.randn(1000) * params.get("param1", 1.0),
            bins=30, color=params.get("color", "blue"), alpha=0.7
        ),
        "params": {"param1": 1.0, "color": "blue"}
    }
}

In [6]:
display_app_layout()

In [7]:
# Imports and Initial Setup
from plot_functions import plot_graph, plot_histogram  # Assuming plot functions are defined
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from PIL import Image

# Initialize global variables
slides = []
selected_slide_index = 0
plot_registry = {
    "plot_graph": {
        "function": plot_graph,
        "params": {"param1": 1.0, "param2": 1.0, "color": "blue"}
    },
    "plot_histogram": {
        "function": plot_histogram,
        "params": {"param1": 1.0, "color": "blue"}
    },
}

# Sidebar, main display, and customization window layout
left_sidebar = widgets.VBox()
main_slide_display = widgets.Output(layout=widgets.Layout(width="60%", border="1px solid black"))
customization_window = widgets.VBox(layout=widgets.Layout(width="20%", border="1px solid black"))

# Define position mappings within the grid
position_coordinates = {
    "top-left": (0.1, 0.6),
    "top-right": (0.6, 0.6),
    "bottom-left": (0.1, 0.1),
    "bottom-right": (0.6, 0.1)
}

# Data Management: Thumbnail Generator
def generate_slide_thumbnail(slide):
    """Generate a thumbnail of the slide's plots as an image."""
    fig, axs = plt.subplots(2, 2, figsize=(3, 3))  # Create a small 2x2 grid for the thumbnail

    # Plot each item on its position within the 2x2 grid
    for plot_data in slide["plots"]:
        ax = axs[0][0] if plot_data["position"] == "top-left" else \
             axs[0][1] if plot_data["position"] == "top-right" else \
             axs[1][0] if plot_data["position"] == "bottom-left" else \
             axs[1][1]  # bottom-right default

        plot_function = plot_registry[plot_data["function_name"]]["function"]
        plot_function(ax, **plot_data["params"])
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()

    # Convert the plot to an image and resize it to thumbnail size
    buf = BytesIO()
    fig.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    img.thumbnail((100, 100))  # Resize to a small thumbnail
    return img

# UI Component: Update Sidebar
def update_sidebar():
    """Update the sidebar to show slide thumbnails for each slide."""
    sidebar_content = []
    for i, slide in enumerate(slides):
        thumbnail_img = generate_slide_thumbnail(slide)

        # Convert thumbnail image to widget
        with BytesIO() as output:
            thumbnail_img.save(output, format="PNG")
            img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)
        
        slide_label = widgets.Label(f"Slide {i + 1}")
        slide_button_box = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
            width="120px", height="150px", border="1px solid black", align_items="center"
        ))

        # Use Button to make the thumbnail selectable
        select_button = widgets.Button(description=f"Select Slide {i + 1}", layout=widgets.Layout(width='100%'))
        select_button.on_click(lambda b, idx=i: select_slide(idx))

        # Combine thumbnail, label, and button
        slide_button = widgets.VBox([slide_button_box, select_button])
        sidebar_content.append(slide_button)

    left_sidebar.children = sidebar_content

# Plotting and Customization Functions
def update_custom_plot(plot_type):
    """Update the plot in the customization window based on current widget values."""
    plot_function = plot_registry[plot_type]["function"]

    # Collect the current parameter values
    param_values = {
        param1_slider.description: param1_slider.value,
        param2_slider.description: param2_slider.value,
        color_picker.description: color_picker.value
    }
    
    # Clear the output widget and display the updated plot
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_function(ax, **param_values)
        plt.show()

def open_customization_window(plot_type="plot_graph"):
    """Displays the customization window with controls and a live-updating plot."""
    global param1_slider, param2_slider, color_picker, plot_output, position_dropdown, plot_type_dropdown

    # Dropdown for plot type
    plot_type_dropdown = widgets.Dropdown(options=list(plot_registry.keys()), description="Plot Type:")
    
    # Dropdown for position selection
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"],
                                         description="Position")

    # Initialize sliders and color picker for parameters
    param1_slider = widgets.FloatSlider(value=plot_registry[plot_type]["params"].get("param1", 1.0), 
                                        min=0.1, max=2.0, step=0.1, description="param1")
    param2_slider = widgets.FloatSlider(value=plot_registry[plot_type]["params"].get("param2", 1.0), 
                                        min=0.1, max=2.0, step=0.1, description="param2")
    color_picker = widgets.ColorPicker(value=plot_registry[plot_type]["params"].get("color", "blue"), description="color")
    plot_output = widgets.Output()

    # Attach observers to dynamically update the plot in the customization window
    param1_slider.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")
    param2_slider.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")
    color_picker.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")
    plot_type_dropdown.observe(lambda change: update_custom_plot(plot_type_dropdown.value), names="value")

    # Initial plot rendering
    update_custom_plot(plot_type)

    # Confirm button to add plot to slide
    confirm_button = widgets.Button(description="Confirm Plot Addition")
    confirm_button.on_click(lambda b: add_plot_to_slide(position_dropdown.value))

    # Display the customization window with sliders, color picker, position dropdown, and live plot
    customization_window.children = [plot_type_dropdown, position_dropdown, param1_slider, param2_slider, color_picker, plot_output, confirm_button]

# Plot Management: Adding and Rendering
def add_plot_to_slide(position="top-left"):
    plot_type = plot_type_dropdown.value
    params = {
        "param1": param1_slider.value,
        "param2": param2_slider.value,
        "color": color_picker.value
    }
    new_plot = {
        "function_name": plot_type,
        "params": params,
        "position": position
    }
    slides[selected_slide_index]["plots"].append(new_plot)
    render_slide()

def render_slide():
    main_slide_display.clear_output(wait=True)
    with main_slide_display:
        slide = slides[selected_slide_index]
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        positions = ["top-left", "top-right", "bottom-left", "bottom-right"]
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            plot_function = plot_registry[plot_data["function_name"]]["function"]
            plot_function(ax, **plot_data["params"])
            ax.set_title("Customized Graph")
        plt.tight_layout()
        plt.show()

def select_slide(index):
    global selected_slide_index
    selected_slide_index = index
    render_slide()

# Application Layout
def display_app_layout():
    update_sidebar()
    render_slide()
    open_customization_window()
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))

# Initialize the application with a single slide
slides.append({"uid": "slide_1", "plots": []})
display_app_layout()

# Control buttons
add_slide_button = widgets.Button(description="Add Slide")
add_slide_button.on_click(lambda b: slides.append({"uid": f"slide_{len(slides) + 1}", "plots": []}) or update_sidebar())

add_plot_button = widgets.Button(description="Add Plot")
add_plot_button.on_click(lambda b: open_customization_window())

export_button = widgets.Button(description="Export to PPT")
# Note: Add export functionality here if needed

display(widgets.HBox([add_slide_button, add_plot_button, export_button]))


In [26]:
# Corrected Imports and Initial Setup
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    """
    Creates a DataFrame with a multi-level column index based on the given inputs.
    """
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    """
    Create a custom stats chart based on the provided data.
    """
    fig, ax = plt.subplots(figsize=(4, 3))  # Adjusted to a smaller figure size for preview in the grid
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plot 20th-80th percentile range as bars
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

    # Plot min/max as horizontal lines
    for i, category in enumerate(categories):
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

    # Plot average as triangles
    for i, category in enumerate(categories):
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

    # Plot last values as diamonds
    for i, category in enumerate(categories):
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

    # Customization
    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.set_title("Custom Stats Chart")
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Custom legend
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, frameon=False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    # Widgets for user input
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    # Sidebar and Main Slide Display
    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    # Function to preview the plot
    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        # Plot Preview based on selected plot type
        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Function to add previewed plot to slide
    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    # Layout for customization window
    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    # Full app layout
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

    # Functions to render and select slides
    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            # Clear any existing content in each axis before plotting
            for ax in axs.flatten():
                ax.clear()
            
            for plot_data in slide["plots"]:
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    # Call the function to draw the chart in the respective subplot axis
                    draw_custom_stats_chart(ax, plot_data["data"])
            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        """Draws a custom stats chart in the given axis."""
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        # Plot 20th-80th percentile range as bars
        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

        # Plot min/max as horizontal lines
        for i, category in enumerate(categories):
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

        # Plot average as triangles
        for i, category in enumerate(categories):
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

        # Plot last values as diamonds
        for i, category in enumerate(categories):
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

        # Customization
        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    # Sidebar update for slide selection
    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    # Initial call to set up sidebar and slides
    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [27]:
# Corrected Imports and Initial Setup
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    """
    Creates a DataFrame with a multi-level column index based on the given inputs.
    """
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    """
    Create a custom stats chart based on the provided data.
    """
    fig, ax = plt.subplots(figsize=(4, 3))  # Adjusted to a smaller figure size for preview in the grid
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plot 20th-80th percentile range as bars
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

    # Plot min/max as horizontal lines
    for i, category in enumerate(categories):
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

    # Plot average as triangles
    for i, category in enumerate(categories):
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

    # Plot last values as diamonds
    for i, category in enumerate(categories):
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

    # Customization
    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.set_title("Custom Stats Chart")
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Custom legend
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, frameon=False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    # Widgets for user input
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    # Sidebar and Main Slide Display
    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    # Function to preview the plot
    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        # Plot Preview based on selected plot type
        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Function to add previewed plot to slide
    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()
        update_sidebar()  # Update sidebar to reflect the new plot in the thumbnail

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    # Layout for customization window
    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    # Full app layout
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

    # Functions to render and select slides
    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            # Clear any existing content in each axis before plotting
            for ax in axs.flatten():
                ax.clear()
            
            for plot_data in slide["plots"]:
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])
            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        """Draws a custom stats chart in the given axis."""
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        # Plot 20th-80th percentile range as bars
        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

        # Plot min/max as horizontal lines
        for i, category in enumerate(categories):
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

        # Plot average as triangles
        for i, category in enumerate(categories):
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

        # Plot last values as diamonds
        for i, category in enumerate(categories):
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

        # Customization
        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    # Sidebar update for slide selection
    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    # Initial call to set up sidebar and slides
    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [28]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(4, 3))  # Adjusted to a smaller figure size for preview in the grid
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plotting elements of the stats chart
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    # Customization
    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        add_to_slide_button.disabled = False

    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()
        update_sidebar()

    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            for ax in axs.flatten():
                ax.clear()
            for plot_data in slide["plots"]:
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])
            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for ax in axs.flatten():
            ax.clear()
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            if plot_data["type"] == "Time Series Plot":
                for column in plot_data["data"].columns:
                    ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
            elif plot_data["type"] == "Custom Stats Chart":
                draw_custom_stats_chart(ax, plot_data["data"])
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail((100, 100))
        return img

    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)

In [29]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(4, 3))  # Adjusted to a smaller figure size for preview in the grid
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plotting elements of the stats chart
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    # Customization
    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        add_to_slide_button.disabled = False

    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()
        update_sidebar()

    def on_edit_plot_button_clicked(plot_data):
        """Open the selected plot in the customization view for editing."""
        plot_type_widget.value = plot_data["type"]
        current_plot_data.update(plot_data)
        if plot_data["type"] == "Time Series Plot":
            # Assuming we have some way to get back the x and y data information
            update_custom_plot()

    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            for ax in axs.flatten():
                ax.clear()
            for i, plot_data in enumerate(slide["plots"]):
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])
                # Add a checkbox to the top right of each plot for editing
                edit_checkbox = widgets.Checkbox(description="Edit", value=False)
                edit_checkbox.observe(lambda change, data=plot_data: on_edit_plot_button_clicked(data) if change['new'] else None, names='value')
                ax_checkbox = widgets.HBox([widgets.Label(''), edit_checkbox])
                display(ax_checkbox)
            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for ax in axs.flatten():
            ax.clear()
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            if plot_data["type"] == "Time Series Plot":
                for column in plot_data["data"].columns:
                    ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
            elif plot_data["type"] == "Custom Stats Chart":
                draw_custom_stats_chart(ax, plot_data["data"])
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail((100, 100))
        return img

    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [32]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(4, 3))
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        add_to_slide_button.disabled = False

    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]

        # Remove any existing plot in the selected position
        slides[slide_index]["plots"] = [plot for plot in slides[slide_index]["plots"] if plot["position"] != position]

        # Add the new plot to the selected position
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()
        update_sidebar()

    def on_edit_plot_button_clicked(plot_data):
        """Open the selected plot in the customization view for editing."""
        plot_type_widget.value = plot_data["type"]
        current_plot_data.update(plot_data)
        if plot_data["type"] == "Time Series Plot":
            update_custom_plot()

    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            for ax in axs.flatten():
                ax.clear()

            for i, plot_data in enumerate(slide["plots"]):
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])

                # Add a single checkbox on the top-right for each plot to edit
                checkbox = widgets.Checkbox(value=False, description="Edit", layout=widgets.Layout(width='50px', height='20px'))
                
                def checkbox_handler(change, plot=plot_data):
                    if change['new']:
                        # Load plot data into customization for editing
                        on_edit_plot_button_clicked(plot)

                checkbox.observe(checkbox_handler, names='value')
                display(widgets.HBox([checkbox], layout=widgets.Layout(align_items='flex-end', justify_content='flex-end')))

            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for ax in axs.flatten():
            ax.clear()
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            if plot_data["type"] == "Time Series Plot":
                for column in plot_data["data"].columns:
                    ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
            elif plot_data["type"] == "Custom Stats Chart":
                draw_custom_stats_chart(ax, plot_data["data"])
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail((100, 100))
        return img

    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [33]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(4, 3))
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        add_to_slide_button.disabled = False

    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]

        # Remove any existing plot in the selected position
        slides[slide_index]["plots"] = [plot for plot in slides[slide_index]["plots"] if plot["position"] != position]

        # Add the new plot to the selected position
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()
        update_sidebar()

    def on_edit_plot_button_clicked(plot_data):
        """Open the selected plot in the customization view for editing."""
        plot_type_widget.value = plot_data["type"]
        current_plot_data.update(plot_data)
        if plot_data["type"] == "Time Series Plot":
            update_custom_plot()

    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            for ax in axs.flatten():
                ax.clear()

            for i, plot_data in enumerate(slide["plots"]):
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])

                # Add a single checkbox on the top-right for each plot to edit
                checkbox = widgets.Checkbox(value=False, description="Edit", layout=widgets.Layout(width='50px', height='20px'))
                
                def checkbox_handler(change, plot=plot_data):
                    if change['new']:
                        # Load plot data into customization for editing
                        on_edit_plot_button_clicked(plot)

                checkbox.observe(checkbox_handler, names='value')
                display(checkbox)

            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for ax in axs.flatten():
            ax.clear()
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            if plot_data["type"] == "Time Series Plot":
                for column in plot_data["data"].columns:
                    ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
            elif plot_data["type"] == "Custom Stats Chart":
                draw_custom_stats_chart(ax, plot_data["data"])
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail((100, 100))
        return img

    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [36]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(4, 3))
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()
    return fig

# Step 3: Main app with plotting and slide management
def create_app(df):
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    export_button = widgets.Button(description="Export to PPT", button_style='warning')
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0
    current_plot_data = {}

    left_sidebar = widgets.VBox(layout=widgets.Layout(width="15%", border="1px solid black"))
    main_slide_display = widgets.Output(layout=widgets.Layout(width="50%", height="500px", border="1px solid black"))
    customization_window = widgets.VBox(layout=widgets.Layout(width="35%", border="1px solid black"))

    def on_plot_button_clicked(b):
        nonlocal current_plot_data
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                add_to_slide_button.disabled = True
                return

        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = filtered_df.rolling(window=selected_window).mean().fillna(0)
                fig, ax = plt.subplots(figsize=(5, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": result_df,
                    "type": "Time Series Plot",
                    "position": position_dropdown.value
                }
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
                plt.show()
                current_plot_data = {
                    "figure": fig,
                    "data": filtered_df,
                    "type": "Custom Stats Chart",
                    "position": position_dropdown.value
                }

        add_to_slide_button.disabled = False

    def on_add_to_slide_button_clicked(b):
        if not current_plot_data:
            return
        slide_index = selected_slide_index
        position = current_plot_data["position"]

        # Remove any existing plot in the selected position
        slides[slide_index]["plots"] = [plot for plot in slides[slide_index]["plots"] if plot["position"] != position]

        # Add the new plot to the selected position
        slides[slide_index]["plots"].append({
            "type": current_plot_data["type"],
            "data": current_plot_data["data"],
            "position": position
        })
        render_slide()
        update_sidebar()

    def on_export_button_clicked(b):
        prs = Presentation()
        for slide in slides:
            slide_layout = prs.slide_layouts[5]
            slide_to_add = prs.slides.add_slide(slide_layout)

            for plot_data in slide["plots"]:
                img_stream = BytesIO()
                fig, ax = plt.subplots(figsize=(5, 4))
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])
                fig.savefig(img_stream, format='png')
                img_stream.seek(0)

                slide_to_add.shapes.add_picture(img_stream, Inches(0.5), Inches(0.5), width=Inches(5.0), height=Inches(3.75))
                plt.close(fig)

        prs.save("Generated_Presentation.pptx")
        print("Presentation exported as 'Generated_Presentation.pptx'")

    def on_add_slide_button_clicked(b):
        nonlocal selected_slide_index
        slides.append({"plots": []})
        selected_slide_index = len(slides) - 1
        update_sidebar()
        render_slide()

    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)
    export_button.on_click(on_export_button_clicked)
    add_slide_button = widgets.Button(description="Add Slide", button_style="primary")
    add_slide_button.on_click(on_add_slide_button_clicked)

    customization_window.children = [
        plot_type_widget,
        udl_widget,
        matu_widget,
        param_widget,
        level_widget,
        window_widget,
        start_date_widget,
        end_date_widget,
        position_dropdown,
        plot_button,
        add_to_slide_button,
        output_plot
    ]

    control_buttons = widgets.HBox([add_slide_button, export_button])

    display(widgets.VBox([widgets.HBox([left_sidebar, main_slide_display, customization_window]),
                          control_buttons]))
    slides.append({"plots": []})

    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            for ax in axs.flatten():
                ax.clear()

            for i, plot_data in enumerate(slide["plots"]):
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                if plot_data["type"] == "Time Series Plot":
                    for column in plot_data["data"].columns:
                        ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
                    ax.legend()
                elif plot_data["type"] == "Custom Stats Chart":
                    draw_custom_stats_chart(ax, plot_data["data"])

            plt.tight_layout()
            plt.show()

    def draw_custom_stats_chart(ax, df):
        categories = df.columns.get_level_values('UDL').unique()
        min_values = df.min().values
        max_values = df.max().values
        avg_values = df.mean().values
        last_values = df.iloc[-1].values
        percentile_20 = df.quantile(0.2).values
        percentile_80 = df.quantile(0.8).values

        colors = {
            "percentile_range": "gray",
            "min_max": "black",
            "avg": "green",
            "last": "red"
        }
        icon_width = 0.3

        for i, category in enumerate(categories):
            ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
            ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
            ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

        ax.set_xticks(range(len(categories)))
        ax.set_xticklabels(categories)
        ax.axhline(0, color='black', linewidth=2.0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    def select_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail_img = generate_slide_thumbnail(slide)
            with BytesIO() as output:
                thumbnail_img.save(output, format="PNG")
                img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)

            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
                width="120px", height="150px", border="1px solid black", align_items="center"
            ))

            slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
            slide_button_box.on_click(lambda b, idx=i: select_slide(idx))

            sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

        left_sidebar.children = sidebar_content

    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for ax in axs.flatten():
            ax.clear()
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            if plot_data["type"] == "Time Series Plot":
                for column in plot_data["data"].columns:
                    ax.plot(plot_data["data"].index, plot_data["data"][column], label=str(column))
            elif plot_data["type"] == "Custom Stats Chart":
                draw_custom_stats_chart(ax, plot_data["data"])
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail((100, 100))
        return img

    update_sidebar()

# Example usage with multi-index data
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)
